Graphics including GenomeDiagram {#chapter:graphics}
================================

The `Bio.Graphics` module depends on the third party Python library
[ReportLab](http://www.reportlab.org). Although focused on producing PDF
files, ReportLab can also create encapsulated postscript (EPS) and (SVG)
files. In addition to these vector based images, provided certain
further dependencies such as the [Python Imaging Library
(PIL)](http://www.pythonware.com/products/pil/) are installed, ReportLab
can also output bitmap images (including JPEG, PNG, GIF, BMP and PICT
formats).

GenomeDiagram {#sec:genomediagram}
-------------

### Introduction

The `Bio.Graphics.GenomeDiagram` module was added to Biopython 1.50,
having previously been available as a separate Python module dependent
on Biopython. GenomeDiagram is described in the Bioinformatics journal
publication by Pritchard et al. (2006) @pritchard2006, which includes
some examples images. There is a PDF copy of the old manual here,
<http://biopython.org/DIST/docs/GenomeDiagram/userguide.pdf> which has
some more examples. As the name might suggest, GenomeDiagram was
designed for drawing whole genomes, in particular prokaryotic genomes,
either as linear diagrams (optionally broken up into fragments to fit
better) or as circular wheel diagrams. Have a look at Figure 2 in Toth
*et al.* (2006) @toth2006 for a good example. It proved also well suited
to drawing quite detailed figures for smaller genomes such as phage,
plasmids or mitochrondia, for example see Figures 1 and 2 in Van der
Auwera *et al.* (2009) @vanderauwera2009 (shown with additional manual
editing).

This module is easiest to use if you have your genome loaded as a
`SeqRecord` object containing lots of `SeqFeature` objects - for example
as loaded from a GenBank file (see Chapters \[chapter:SeqRecord\]
and \[chapter:Bio.SeqIO\]).

### Diagrams, tracks, feature-sets and features

GenomeDiagram uses a nested set of objects. At the top level, you have a
diagram object representing a sequence (or sequence region) along the
horizontal axis (or circle). A diagram can contain one or more tracks,
shown stacked vertically (or radially on circular diagrams). These will
typically all have the same length and represent the same sequence
region. You might use one track to show the gene locations, another to
show regulatory regions, and a third track to show the GC percentage.
The most commonly used type of track will contain features, bundled
together in feature-sets. You might choose to use one feature-set for
all your CDS features, and another for tRNA features. This isn’t
required - they can all go in the same feature-set, but it makes it
easier to update the properties of just selected features (e.g. make all
the tRNA features red).

There are two main ways to build up a complete diagram. Firstly, the top
down approach where you create a diagram object, and then using its
methods add track(s), and use the track methods to add feature-set(s),
and use their methods to add the features. Secondly, you can create the
individual objects separately (in whatever order suits your code), and
then combine them.

### A top down example {#sec:gd_top_down}

We’re going to draw a whole genome from a `SeqRecord` object read in
from a GenBank file (see Chapter \[chapter:Bio.SeqIO\]). This example
uses the pPCP1 plasmid from *Yersinia pestis biovar Microtus*, the file
is included with the Biopython unit tests under the GenBank folder, or
online
[`NC_005816.gb`](http://biopython.org/SRC/biopython/Tests/GenBank/NC_005816.gb)
from our website.




We’re using a top down approach, so after loading in our sequence we
next create an empty diagram, then add an (empty) track, and to that add
an (empty) feature set:




Now the fun part - we take each gene `SeqFeature` object in our
`SeqRecord`, and use it to generate a feature on the diagram. We’re
going to color them blue, alternating between a dark blue and a light
blue.




Now we come to actually making the output file. This happens in two
steps, first we call the `draw` method, which creates all the shapes
using ReportLab objects. Then we call the `write` method which renders
these to the requested file format. Note you can output in multiple file
formats:




Also, provided you have the dependencies installed, you can also do
bitmaps, for example:




The expected output is shown in Figure \[fig:plasmid\_linear\].

![Simple linear diagram for *Yersinia pestis biovar Microtus* plasmid
pPCP1.<span
data-label="fig:plasmid_linear"></span>](images/plasmid_linear.png)

Notice that the `fragments` argument which we set to four controls how
many pieces the genome gets broken up into.

If you want to do a circular figure, then try this:




The expected output is shown in Figure \[fig:plasmid\_circular\].

![Simple circular diagram for *Yersinia pestis biovar Microtus* plasmid
pPCP1.<span
data-label="fig:plasmid_circular"></span>](images/plasmid_circular.png)

These figures are not very exciting, but we’ve only just got started.

### A bottom up example

Now let’s produce exactly the same figures, but using the bottom up
approach. This means we create the different objects directly (and this
can be done in almost any order) and then combine them.




You can now call the `draw` and `write` methods as before to produce a
linear or circular diagram, using the code at the end of the top-down
example above. The figures should be identical.

### Features without a SeqFeature {#sec:gd_features_without_seqfeatures}

In the above example we used a `SeqRecord`’s `SeqFeature` objects to
build our diagram (see also Section \[sec:seq\_features\]). Sometimes
you won’t have `SeqFeature` objects, but just the coordinates for a
feature you want to draw. You have to create minimal `SeqFeature`
object, but this is easy:




For strand, use `+1` for the forward strand, `-1` for the reverse
strand, and `None` for both. Here is a short self contained example:




The top part of the image in the next subsection shows the output

The output is shown at the top of Figure \[fig:gd\_sigil\_labels\]

(in the default feature color, pale green).

Notice that we have used the `name` argument here to specify the caption
text for these features. This is discussed in more detail next.

### Feature captions {#sec:gd_feature_captions}

Recall we used the following (where `feature` was a `SeqFeature` object)
to add a feature to the diagram:




In the example above the `SeqFeature` annotation was used to pick a
sensible caption for the features. By default the following possible
entries under the `SeqFeature` object’s qualifiers dictionary are used:
`gene`, `label`, `name`, `locus_tag`, and `product`. More simply, you
can specify a name directly:




In addition to the caption text for each feature’s label, you can also
choose the font, position (this defaults to the start of the sigil, you
can also choose the middle or at the end) and orientation (for linear
diagrams only, where this defaults to rotated by $45$ degrees):




Combining each of these three fragments with the complete example in the
previous section should give something like

this:

\[fig:gd\_sigil\_labels\]

the tracks in Figure \[fig:gd\_sigil\_labels\].

![Simple GenomeDiagram showing label options. The top plot in pale green
shows the default label settings (see
Section \[sec:gd\_features\_without\_seqfeatures\]) while the rest show
variations in the label size, position and orientation (see
Section \[sec:gd\_feature\_captions\]). <span
data-label="fig:gd_sigil_labels"></span>](images/GD_sigil_labels.png)

We’ve not shown it here, but you can also set `label_color` to control
the label’s color (used in Section \[sec:gd\_nice\_example\]).

You’ll notice the default font is quite small - this makes sense because
you will usually be drawing many (small) features on a page, not just a
few large ones as shown here.

### Feature sigils {#sec:gd_sigils}

The examples above have all just used the default sigil for the feature,
a plain box, which was all that was available in the last publicly
released standalone version of GenomeDiagram. Arrow sigils were included
when GenomeDiagram was added to Biopython 1.50:




Biopython 1.61 added three more sigils,




These are shown

below.

in Figure \[fig:gd\_sigils\].

Most sigils fit into a bounding box (as given by the default BOX sigil),
either above or below the axis for the forward or reverse strand, or
straddling it (double the height) for strand-less features. The BIGARROW
sigil is different, always straddling the axis with the direction taken
from the feature’s stand.

![Simple GenomeDiagram showing different sigils (see
Section \[sec:gd\_sigils\])<span
data-label="fig:gd_sigils"></span>](images/GD_sigils.png)

### Arrow sigils {#sec:gd_arrow_sigils}

We introduced the arrow sigils in the previous section. There are two
additional options to adjust the shapes of the arrows, firstly the
thickness of the arrow shaft, given as a proportion of the height of the
bounding box:




The results are shown below:

The results are shown in Figure \[fig:gd\_sigil\_arrow\_shafts\].

![Simple GenomeDiagram showing arrow shaft options (see
Section \[sec:gd\_arrow\_sigils\])<span
data-label="fig:gd_sigil_arrow_shafts"></span>](images/GD_sigil_arrow_shafts.png)

Secondly, the length of the arrow head - given as a proportion of the
height of the bounding box (defaulting to $0.5$, or $50\%$):




The results are shown below:

The results are shown in Figure \[fig:gd\_sigil\_arrow\_heads\].

![Simple GenomeDiagram showing arrow head options (see
Section \[sec:gd\_arrow\_sigils\])<span
data-label="fig:gd_sigil_arrow_heads"></span>](images/GD_sigil_arrow_heads.png)

Biopython 1.61 adds a new `BIGARROW` sigil which always stradles the
axis, pointing left for the reverse strand or right otherwise:




All the shaft and arrow head options shown above for the `ARROW` sigil
can be used for the `BIGARROW` sigil too.

### A nice example {#sec:gd_nice_example}

Now let’s return to the pPCP1 plasmid from *Yersinia pestis biovar
Microtus*, and the top down approach used in
Section \[sec:gd\_top\_down\], but take advantage of the sigil options
we’ve now discussed. This time we’ll use arrows for the genes, and
overlay them with strand-less features (as plain boxes) showing the
position of some restriction digest sites.




And the output:

The expected output is shown in Figures \[fig:plasmid\_linear\_nice\]
and \[fig:plasmid\_circular\_nice\].

![Linear diagram for *Yersinia pestis biovar Microtus* plasmid pPCP1
showing selected restriction digest sites (see
Section \[sec:gd\_nice\_example\]).<span
data-label="fig:plasmid_linear_nice"></span>](images/plasmid_linear_nice.png)

![Circular diagram for *Yersinia pestis biovar Microtus* plasmid pPCP1
showing selected restriction digest sites (see
Section \[sec:gd\_nice\_example\]).<span
data-label="fig:plasmid_circular_nice"></span>](images/plasmid_circular_nice.png)

### Multiple tracks {#sec:gd_multiple_tracks}

All the examples so far have used a single track, but you can have more
than one track – for example show the genes on one, and repeat regions
on another. In this example we’re going to show three phage genomes side
by side to scale, inspired by Figure 6 in Proux <span>*e*t al.</span>
(2002) @proux2002. We’ll need the GenBank files for the following three
phage:

-   `NC_002703` – Lactococcus phage Tuc2009, complete genome
    ($38347$ bp)

-   `AF323668` – Bacteriophage bIL285, complete genome ($35538$ bp)

-   `NC_003212` – *Listeria innocua* Clip11262, complete genome, of
    which we are focussing only on integrated prophage 5
    (similar length).

You can download these using Entrez if you like, see
Section \[sec:efetch\] for more details. For the third record we’ve
worked out where the phage is integrated into the genome, and slice the
record to extract it (with the features preserved, see
Section \[sec:SeqRecord-slicing\]), and must also reverse complement to
match the orientation of the first two phage (again preserving the
features, see Section \[sec:SeqRecord-reverse-complement\]):




The figure we are imitating used different colors for different gene
functions. One way to do this is to edit the GenBank file to record
color preferences for each feature - something [Sanger’s Artemis
editor](http://www.sanger.ac.uk/resources/software/artemis/) does, and
which GenomeDiagram should understand. Here however, we’ll just hard
code three lists of colors.

Note that the annotation in the GenBank files doesn’t exactly match that
shown in Proux *et al.*, they have drawn some unannotated genes.




Now to draw them – this time we add three tracks to the diagram, and
also notice they are given different start/end values to reflect their
different lengths (this requires Biopython 1.59 or later).




The result:

The expected output is shown in Figure \[fig:three\_track\_simple\].

![Linear diagram with three tracks for Lactococcus phage Tuc2009
(NC\_002703), bacteriophage bIL285 (AF323668), and prophage 5 from
*Listeria innocua* Clip11262 (NC\_003212) (see
Section \[sec:gd\_multiple\_tracks\]).<span
data-label="fig:three_track_simple"></span>](images/three_track_simple.png)

I did wonder why in the original manuscript there were no red or orange
genes marked in the bottom phage. Another important point is here the
phage are shown with different lengths - this is because they are all
drawn to the same scale (they *are* different lengths).

The key difference from the published figure is they have color-coded
links between similar proteins – which is what we will do in the next
section.

### Cross-Links between tracks {#sec:gd_cross_links}

Biopython 1.59 added the ability to draw cross links between tracks -
both simple linear diagrams as we will show here, but also linear
diagrams split into fragments and circular diagrams.

Continuing the example from the previous section inspired by Figure 6
from Proux *et al.* 2002 @proux2002, we would need a list of cross links
between pairs of genes, along with a score or color to use.
Realistically you might extract this from a BLAST file computationally,
but here I have manually typed them in.

My naming convention continues to refer to the three phage as A, B and
C. Here are the links we want to show between A and B, given as a list
of tuples (percentage similarity score, gene in A, gene in B).




Likewise for B and C:




For the first and last phage these identifiers are locus tags, for the
middle phage there are no locus tags so I’ve used gene names instead.
The following little helper function lets us lookup a feature using
either a locus tag or gene name:




We can now turn those list of identifier pairs into SeqFeature pairs,
and thus find their location co-ordinates. We can now add all that code
and the following snippet to the previous example (just before the
`gd_diagram.draw(...)` line – see the finished example script
[Proux\_et\_al\_2002\_Figure\_6.py](http://biopython.org/SRC/biopython/Doc/examples/Proux_et_al_2002_Figure_6.py)
included in the `Doc/examples` folder of the Biopython source code) to
add cross links to the figure:




There are several important pieces to this code. First the
`GenomeDiagram` object has a `cross_track_links` attribute which is just
a list of `CrossLink` objects. Each `CrossLink` object takes two sets of
track-specific co-ordinates (here given as tuples, you can alternatively
use a `GenomeDiagram.Feature` object instead). You can optionally supply
a colour, border color, and say if this link should be drawn flipped
(useful for showing inversions).

You can also see how we turn the BLAST percentage identity score into a
colour, interpolating between white ($0\%$) and a dark red ($100\%$). In
this example we don’t have any problems with overlapping cross-links.
One way to tackle that is to use transparency in ReportLab, by using
colors with their alpha channel set. However, this kind of shaded color
scheme combined with overlap transparency would be difficult to
interpret.

The result:

The expected output is shown in Figure \[fig:three\_track\_cl\].

![Linear diagram with three tracks for Lactococcus phage Tuc2009
(NC\_002703), bacteriophage bIL285 (AF323668), and prophage 5 from
*Listeria innocua* Clip11262 (NC\_003212) plus basic cross-links shaded
by percentage identity (see Section \[sec:gd\_cross\_links\]).<span
data-label="fig:three_track_cl"></span>](images/three_track_cl.png)

There is still a lot more that can be done within Biopython to help
improve this figure. First of all, the cross links in this case are
between proteins which are drawn in a strand specific manor. It can help
to add a background region (a feature using the ‘BOX’ sigil) on the
feature track to extend the cross link. Also, we could reduce the
vertical height of the feature tracks to allocate more to the links
instead – one way to do that is to allocate space for empty tracks.
Furthermore, in cases like this where there are no large gene overlaps,
we can use the axis-straddling `BIGARROW` sigil, which allows us to
further reduce the vertical space needed for the track. These
improvements are demonstrated in the example script
[Proux\_et\_al\_2002\_Figure\_6.py](http://biopython.org/SRC/biopython/Doc/examples/Proux_et_al_2002_Figure_6.py)
included in the `Doc/examples` folder of the Biopython source code.

The result:

The expected output is shown in Figure \[fig:three\_track\_cl2\].

![Linear diagram with three tracks for Lactococcus phage Tuc2009
(NC\_002703), bacteriophage bIL285 (AF323668), and prophage 5 from
*Listeria innocua* Clip11262 (NC\_003212) plus cross-links shaded by
percentage identity (see Section \[sec:gd\_cross\_links\]).<span
data-label="fig:three_track_cl2"></span>](images/three_track_cl2a.png)

Beyond that, finishing touches you might want to do manually in a vector
image editor include fine tuning the placement of gene labels, and
adding other custom annotation such as highlighting particular regions.

Although not really necessary in this example since none of the
cross-links overlap, using a transparent color in ReportLab is a very
useful technique for superimposing multiple links. However, in this case
a shaded color scheme should be avoided.

### Further options

You can control the tick marks to show the scale – after all every graph
should show its units, and the number of the grey-track labels.

Also, we have only used the `FeatureSet` so far. GenomeDiagram also has
a `GraphSet` which can be used for show line graphs, bar charts and heat
plots (e.g. to show plots of GC% on a track parallel to the features).

These options are not covered here yet, so for now we refer you to the
[\
](http://biopython.org/DIST/docs/GenomeDiagram/userguide.pdf)<span>User
Guide (PDF)</span> included with the standalone version of GenomeDiagram
(but please read the next section first), and the docstrings.

### Converting old code

If you have old code written using the standalone version of
GenomeDiagram, and you want to switch it over to using the new version
included with Biopython then you will have to make a few changes - most
importantly to your import statements.

Also, the older version of GenomeDiagram used only the UK spellings of
color and center (colour and centre). You will need to change to the
American spellings, although for several years the Biopython version of
GenomeDiagram supported both.

For example, if you used to have:




you could just switch the import statements like this:




and hopefully that should be enough. In the long term you might want to
switch to the new names, but you would have to change more of your code:




or:




If you run into difficulties, please ask on the Biopython mailing list
for advice. One catch is that we have not included the old module
`GenomeDiagram.GDUtilities` yet. This included a number of GC% related
functions, which will probably be merged under `Bio.SeqUtils` later on.

Chromosomes
-----------

The `Bio.Graphics.BasicChromosome` module allows drawing of chromosomes.
There is an example in Jupe *et al.* (2012) @jupe2012 (open access)
using colors to highlight different gene families.

### Simple Chromosomes

Here is a very simple example - for which we’ll use *Arabidopsis
thaliana*.

![Simple chromosome diagram for *Arabidopsis thaliana*.<span
data-label="fig:simplechromosome"></span>](images/simple_chrom.pdf)

![Chromosome diagram for *Arabidopsis thaliana* showing tRNA genes.<span
data-label="fig:trnachromosome"></span>](images/tRNA_chrom.pdf)

You can skip this bit, but first I downloaded the five sequenced
chromosomes from the NCBI’s FTP site
<ftp://ftp.ncbi.nlm.nih.gov/genomes/Arabidopsis_thaliana> and then
parsed them with `Bio.SeqIO` to find out their lengths. You could use
the GenBank files for this, but it is faster to use the FASTA files for
the whole chromosomes:




This gave the lengths of the five chromosomes, which we’ll now use in
the following short demonstration of the `BasicChromosome` module:




This should create a very simple PDF file, shown

here:

in Figure \[fig:simplechromosome\].

This example is deliberately short and sweet. The next example shows the
location of features of interest.

### Annotated Chromosomes

Continuing from the previous example, let’s also show the tRNA genes.
We’ll get their locations by parsing the GenBank files for the five
*Arabidopsis thaliana* chromosomes. You’ll need to download these files
from the NCBI FTP site
<ftp://ftp.ncbi.nlm.nih.gov/genomes/Arabidopsis_thaliana>, and preserve
the subdirectory names or edit the paths below:

